In [543]:
import lyricsgenius
import pandas as pd
import numpy as np
import re
from textblob import TextBlob, Word
from textblob import TextBlob, Word
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

import collections
from collections import Counter

import pageviewapi
import wikipedia


1) Using Genius Api search for your favorite 3 artists.

    a) Get 5 most popular songs’ lyrics for each of them. ( max_songs = 5)
    b) Make dataframe with columns “Artist”,”Song”,”Lyrics”,”Year”,”Sentiment_Score”
    c) What are the most positive and negative songs?

In [167]:
Joe_Cocker = genius.search_artist("Joe_Cocker", max_songs=5, sort="title")

Searching for songs by Joe_Cocker...

Changing artist name to 'Joe Cocker'
Song 1: "Across From Midnight"
Song 2: "A Girl Like You"
Song 3: "Ain’t Gonna Cry Again"
Song 4: "Ain’t No Sunshine"
Song 5: "(All I Know) Feels Like Forever"

Reached user-specified song limit (5).
Done. Found 5 songs.


In [172]:
Frank_Sinatra = genius.search_artist("Frank_Sinatra", max_songs=5, sort="title")

Searching for songs by Frank_Sinatra...

Changing artist name to 'Frank Sinatra'
Song 1: "A Baby Just Like You"
Song 2: "A Day In The Life Of A Fool"
Song 3: "Adeste Fideles"
Song 4: "Adeste Fideles (O, Come All Ye Faithful)"
Song 5: "A Fellow Needs a Girl"

Reached user-specified song limit (5).
Done. Found 5 songs.


In [176]:
Johnny_Cash = genius.search_artist("Johnny_Cash", max_songs=5, sort="title")

Searching for songs by Johnny_Cash...

Changing artist name to 'Johnny Cash'
Song 1: "1 Corinthians 15:55"
Song 2: "25 Minutes To Go"
Song 3: "25 Minutes to Go (v2)"
Song 4: "40 Shades of Green"
Song 5: "Abner Brown"

Reached user-specified song limit (5).
Done. Found 5 songs.


In [234]:
Artist_info = np.array([[Joe_Cocker.songs[i].artist for i in range(len(Joe_Cocker.songs))] +\
                 [Frank_Sinatra.songs[i].artist for i in range(len(Frank_Sinatra.songs))]+\
                 [Johnny_Cash.songs[i].artist for i in range(len(Johnny_Cash.songs))],\
        #artists' names and combined
                 [Joe_Cocker.songs[i].title for i in range(len(Joe_Cocker.songs))]+\
                 [Frank_Sinatra.songs[i].title for i in range(len(Frank_Sinatra.songs))]+\
                 [Johnny_Cash.songs[i].title for i in range(len(Johnny_Cash.songs))],\
        #artists' song titles respectively and combined
                [Joe_Cocker.songs[i].lyrics.replace("\n"," ") for i in range(len(Joe_Cocker.songs))]+\
                 [Frank_Sinatra.songs[i].lyrics.replace("\n"," ") for i in range(len(Frank_Sinatra.songs))]+\
                 [Johnny_Cash.songs[i].lyrics.replace("\n"," ") for i in range(len(Johnny_Cash.songs))],\
        #artists' song lyrics, replaced notation of "\n" with space respectively and combined
                [Joe_Cocker.songs[i].year for i in range(len(Joe_Cocker.songs))]+\
                 [Frank_Sinatra.songs[i].year for i in range(len(Frank_Sinatra.songs))]+\
                 [Johnny_Cash.songs[i].year for i in range(len(Johnny_Cash.songs))],\
        #comprehension I got artists' song yeasr respectively, but I got actual result only for one song
                [TextBlob(Joe_Cocker.songs[i].lyrics.replace("\n"," ")).sentiment[0] for i in range(len(Joe_Cocker.songs))]+\
                 [TextBlob(Frank_Sinatra.songs[i].lyrics.replace("\n"," ")).sentiment[0] for i in range(len(Frank_Sinatra.songs))]+\
                 [TextBlob(Johnny_Cash.songs[i].lyrics.replace("\n"," ")).sentiment[0] for i in range(len(Johnny_Cash.songs))],\
        #artists' song lyrics sentiment score respectively and combined
               ])

In [235]:
Artist_info_df = pd.DataFrame({"Artist": Artist_info[0], "Song": Artist_info[1],\
              "Lyrics":Artist_info[2],"Year":Artist_info[3],"Sentiment_Score":Artist_info[4]})
#transformation into Pandas Dataframe

In [236]:
Artist_info_df

,Artist,Song,Lyrics,Year,Sentiment_Score
0,Joe Cocker,Across From Midnight,It’s come down to just the two of us Me and my...,None,-0.0450617
1,Joe Cocker,A Girl Like You,"Uh Mmm yeah Uh Uh Uh, ooh oow Yo, the game is...",None,0.0313867
2,Joe Cocker,Ain’t Gonna Cry Again,You're a mystery to me You make your own reali...,None,0.262037
3,Joe Cocker,Ain’t No Sunshine,Ain't no sunshine when she's gone It's not war...,None,-0.1375
4,Joe Cocker,(All I Know) Feels Like Forever,I saw you That was all I had to see Wanted you...,None,0.485714
5,Frank Sinatra,A Baby Just Like You,The season is upon us now A time for gifts and...,None,0.13548
6,Frank Sinatra,A Day In The Life Of A Fool,A day in the life of a fool A sad and a long l...,None,0.0738095
7,Frank Sinatra,Adeste Fideles,"Adeste fideles Laeti triumphantes Venite, veni...",None,0
8,Frank Sinatra,"Adeste Fideles (O, Come All Ye Faithful)","Adeste fideles, laete triumphantes Venite, ven...",None,0.4
9,Frank Sinatra,A Fellow Needs a Girl,"A fellow needs a girl to sit by his side, at t...",None,0.397421


In [237]:
positive = np.array(Artist_info_df.Sentiment_Score).max() #Most positive song is Joe Cockers "(All I Know) Feels Like Forever" song
print(positive)

0.48571428571428577


In [238]:
negative = np.array(Artist_info_df.Sentiment_Score).min() #Most negative song is Joe Cockers "Ain’t No Sunshine" song
print(negative)

-0.1375


2) Using Wikipedia api

    a) Get summary of wikipedia pages of the 3 artists above in russian (set language to russian) .
    b) What are the most common words in summary for each of them?

In [228]:
wikipedia.set_lang("ru")

In [248]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hakob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [251]:
badwords = [ u'я', u'а', u'да', u'но', u'тебе', u'мне', u'ты', u'и', u'у', u'на', u'ща', u'ага',
u'так', u'там', u'какие', u'который', u'какая', u'туда', u'давай', u'короче', u'кажется', u'вообще',
u'ну', u'не', u'чет', u'неа', u'свои', u'наше', u'хотя', u'такое', u'например', u'кароч', u'как-то',
u'нам', u'хм', u'всем', u'нет', u'да', u'оно', u'своем', u'про', u'вы', u'м', u'тд',
u'вся', u'кто-то', u'что-то', u'вам', u'это', u'эта', u'эти', u'этот', u'прям', u'либо', u'как', u'мы',
u'просто', u'блин', u'очень', u'самые', u'твоем', u'ваша', u'кстати', u'вроде', u'типа', u'пока', u'ок']
#dowloaded from https://github.com/nltk/nltk/issues/1367

In [268]:
russian_stopwords = stopwords.words("russian") # from https://www.kaggle.com/alxmamaev/how-to-easy-preprocess-russian-text

In [333]:
stops = russian_stopwords + badwords + ['В',",","``","\''","’",":","'s","‘","–",'—',"$","\'",'/','.','«','»','===','==','(',')',';','@']
#after code execution I manually added some simbols for exclusion

In [334]:
russian_stopwords = list(set(stops))

In [420]:
Joe_Cocker_wiki = wikipedia.page("Joe Cocker").content

In [421]:
Joe_Cocker_wiki_words = nltk.word_tokenize(Joe_Cocker_wiki)

In [422]:
Joe_Cocker_wiki_clean=[] 
for i in Joe_Cocker_wiki_words: 
    if i not in russian_stopwords:
        Joe_Cocker_wiki_clean.append(i)

In [423]:
counter = collections.Counter(Joe_Cocker_wiki_clean) 
print(counter.most_common(50))   #most  popular 50 words

[('Кокер', 11), ('году', 9), ('Джо', 6), ('года', 6), ('My', 5), ('Кокера', 5), ('премию', 4), ('годы', 4), ('англ', 3), ('Cocker', 3), ('2014', 3), ('Офицер', 3), ('Up', 3), ('Where', 3), ('We', 3), ('Belong', 3), ('также', 3), ('одним', 3), ('Rolling', 3), ('The', 3), ('Band', 3), ('With', 3), ('a', 3), ('Little', 3), ('Help', 3), ('from', 3), ('Friends', 3), ('Mad', 3), ('Dogs', 3), ('&', 3), ('Englishmen', 3), ('Joe', 2), ('22', 2), ('декабря', 2), ('Кроуфорд', 2), ('Колорадо', 2), ('британский', 2), ('хриплый', 2), ('баритон', 2), ('репертуара', 2), ('блюзовые', 2), ('the', 2), ('You', 2), ('фильме', 2), ('джентльмен', 2), ('Оскар', 2), ('песня', 2), ('Грэмми', 2), ('исполнителей', 2), ('Stone', 2)]


In [424]:
Frank_Sinatra_wiki = wikipedia.page("Frank_Sinatra").content

In [425]:
Frank_Sinatra_wiki_words = nltk.word_tokenize(Frank_Sinatra_wiki)

In [426]:
Frank_Sinatra_wiki_clean=[] 
for i in Frank_Sinatra_wiki_words: 
    if i not in russian_stopwords:
        Frank_Sinatra_wiki_clean.append(i)

In [427]:
counter = collections.Counter(Frank_Sinatra_wiki_clean) 
print(counter.most_common(50))   #most  popular 50 words

[('Синатра', 38), ('The', 28), ('году', 27), ('the', 19), ('Синатры', 18), ('Sinatra', 11), ('англ', 10), ('Фрэнк', 10), ('Фрэнка', 9), ('лет', 9), ('года', 8), ('1962', 8), ('стал', 7), ('певца', 7), ('контракт', 7), ('Records', 7), ('Дорси', 6), ('Джонни', 6), ('альбом', 6), ('Frank', 6), ('Нэнси', 6), ('1960', 6), ('стали', 5), ('записал', 5), ('США', 5), ('Долли', 5), ('Мартин', 5), ('С', 5), ('также', 5), ('Of', 5), ('Гарднер', 5), ('!', 5), ('A', 5), ('With', 5), ('Me', 5), ('1959', 5), ('to', 5), ('Man', 5), ('1955', 5), ('1965', 5), ('Тони', 5), ('музыкального', 4), ('Оскар', 4), ('1954', 4), ('жизнь', 4), ('золотой', 4), ('мать', 4), ('вместе', 4), ('первый', 4), ('1948', 4)]


In [428]:
Johnny_Cash_wiki = wikipedia.page("Johnny_Cash").content

In [429]:
Johnny_Cash_wiki_words = nltk.word_tokenize(Johnny_Cash_wiki)

In [430]:
Johnny_Cash_wiki_clean=[] 
for i in Johnny_Cash_wiki_words: 
    if i not in russian_stopwords:
        Johnny_Cash_wiki_clean.append(i)

In [431]:
counter = collections.Counter(Johnny_Cash_wiki_clean) 
print(counter.most_common(50))   #most  popular 50 words

[('Кэш', 33), ('Кэша', 24), ('года', 23), ('Джонни', 21), ('году', 16), ('Джун', 16), ('Картер', 13), ('The', 8), ('American', 8), ('кантри', 7), ('Sun', 7), ('Records', 7), ('годы', 7), ('2003', 6), ('своей', 6), ('штат', 6), ('альбом', 6), ('лет', 5), ('Вивиан', 5), ('Folsom', 5), ('получил', 5), ('3', 5), ('июля', 5), ('время', 5), ('четыре', 5), ('Johnny', 4), ('Cash', 4), ('Man', 4), ('Джон', 4), ('записи', 4), ('Prison', 4), ('альбомов', 4), ('стала', 4), ('также', 4), ('стиле', 4), ('смерти', 4), ('музыканта', 4), ('Hurt', 4), ('На', 4), ('февраля', 3), ('самых', 3), ('века', 3), ('рокабилли', 3), ('С', 3), ('Кэшем', 3), ('песни', 3), ('рядом', 3), ('США', 3), ('познакомился', 3), ('женой', 3)]


3) Create a class of a human that has data attribute "nationality" as Armenian (i.e. the general variable).

    a) Make sure that the instances of human class have arguments name, surname, profession and salary (inside init).
    b) The class defined must have a method that returns an expression in this format: "Hi, I am "name" ,"surname". I am a/an "profession" and I earn "salary" a month." (inside the quote values such as name and surname the relevant attributes of that person should be used.
    c) Create one or more objects of the class and execute the class method defined on point b).

In [536]:
class human:
    nationality = "Armenian"
    def __init__(self,name,surname,profession,salary):
        self.name = name
        self.surname = surname
        self.profession = profession
        self.salary = salary       
    def about_human(self):
        print("Hi, I am " + self.name + " " + self.surname + "." + " I am a/an "+ \
              self.profession + " and I earn " + self.salary + " a month.")              


In [537]:
Person_1 = human(name="Hakob", surname="Hakobyan", profession="Economist", salary="Some_amount")

In [538]:
Person_1.about_human()

Hi, I am Hakob Hakobyan. I am a/an Economist and I earn Some_amount a month.


In [539]:
Person_2 = human(name="Gagik", surname="Karapetyan", profession="Taecher", salary="AMD 120,000")

In [540]:
Person_2.about_human()

Hi, I am Gagik Karapetyan. I am a/an Taecher and I earn AMD 120,000 a month.


In [541]:
Person_3 = human(name="Karine", surname="Araqelyan", profession="Data Scientist", salary="AMD 300,000")

In [542]:
Person_3.about_human()

Hi, I am Karine Araqelyan. I am a/an Data Scientist and I earn AMD 300,000 a month.
